In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, BatchNormalization, Add, AveragePooling2D, Input, ReLU
from keras.utils import to_categorical
from tensorflow import Tensor
from keras.callbacks import ModelCheckpoint

In [2]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [3]:
train_x.shape

(50000, 32, 32, 3)

In [4]:
train_x = np.reshape(train_x, [train_x.shape[0], train_x.shape[1], train_x.shape[2], train_x.shape[3]])
test_x = np.reshape(test_x, [test_x.shape[0], test_x.shape[1], test_x.shape[2], test_x.shape[3]])

In [5]:
train_x = train_x.astype('float') / 255
test_x = test_x.astype('float') / 255
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

In [6]:
def relu_bn(inputs:Tensor) -> Tensor:
  re = ReLU()(inputs)
  bn = BatchNormalization()(re)
  return bn

In [7]:
def residual_block(x:Tensor, filters:int, downsampling:bool, kernel_size:int=3) -> Tensor:
  y = Conv2D(filters=filters,
             kernel_size=kernel_size,
             strides=(1 if not downsampling else 2),
             padding = 'same'
             )(x)
  y = relu_bn(y)
  y = Conv2D(filters=filters,
             kernel_size=kernel_size,
             strides=1,
             padding='same'
             )(y)
  if downsampling:
    x = Conv2D(kernel_size=1,
               strides=2,
               filters=filters,
               padding='same')(x)
  out = tf.keras.layers.Add()([x, y])
  out = relu_bn(out)
  return out

In [8]:
def create_res_net():
  inputs = Input(shape=(32,32,3))
  num_filters=64
  num_blocks_list = [2,5,5,2]
  t = BatchNormalization()(inputs)
  t = Conv2D(kernel_size=3,
             filters=num_filters,
             padding='same',
             strides=1)(t)
  t = relu_bn(t)

  for i in range(len(num_blocks_list)):
    num_blocks = num_blocks_list[i]
    for j in range(num_blocks):
      t = residual_block(t,
                         downsampling=(j==0 and i!=0),
                         kernel_size=3,
                         filters=num_filters)
    num_filters *= 2
  
  t = AveragePooling2D(4)(t)
  t = Flatten()(t)
  outputs = Dense(10, activation='softmax')(t)
  model = Model(inputs, outputs)

  model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

In [9]:
model = create_res_net()
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   1792        batch_normalization[0][0]        
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 32, 32, 64)   0           conv2d[0][0]                     
_______________________________________________________________________________________

In [10]:
m = ModelCheckpoint('myfile.h5', monitor='val_loss', save_best_only=True)
model.fit(train_x, train_y, epochs=20, batch_size=500, validation_data=(test_x, test_y)) 

Epoch 1/20
100/100 [==============================] - 32s 322ms/step - loss: 2.4408 - accuracy: 0.1760 - val_loss: 59442.1250 - val_accuracy: 0.1005
Epoch 2/20
100/100 [==============================] - 32s 318ms/step - loss: 1.8720 - accuracy: 0.2855 - val_loss: 22.3753 - val_accuracy: 0.1015
Epoch 3/20
100/100 [==============================] - 32s 318ms/step - loss: 1.7227 - accuracy: 0.3438 - val_loss: 3.3894 - val_accuracy: 0.1836
Epoch 4/20
100/100 [==============================] - 32s 319ms/step - loss: 1.5817 - accuracy: 0.3986 - val_loss: 2.7969 - val_accuracy: 0.2318
Epoch 5/20
100/100 [==============================] - 32s 318ms/step - loss: 1.4731 - accuracy: 0.4451 - val_loss: 2.4880 - val_accuracy: 0.3060
Epoch 6/20
100/100 [==============================] - 32s 318ms/step - loss: 1.3663 - accuracy: 0.4877 - val_loss: 2.0770 - val_accuracy: 0.3293
Epoch 7/20
100/100 [==============================] - 32s 319ms/step - loss: 1.2537 - accuracy: 0.5382 - val_loss: 1.7605 - v